In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torch.nn.functional as F
%matplotlib inline

In [103]:
resnet50 = models.resnet50(pretrained=True)

In [104]:
# for param in resnet50.parameters():
#     param.requires_grad = False
# fc_num_in = resnet50.fc.in_features
# print(fc_num_in)
#Add more linear layers after resnet50
class resnet50_3FC(nn.Module):
    def __init__(self, resnet):
        super(resnet50_3FC, self).__init__()
        self.resnet = resnet
        self.linear1 = nn.Linear(2048, 512)
        self.resnet.fc = self.linear1
        self.fc1_drop = nn.Dropout(p=0.1)
#         self.linear2 = nn.Linear(1024, 256)
#         self.fc2_drop = nn.Dropout(p=0.1)
        self.linear3 = nn.Linear(512, 144)
        
    def forward(self, x):
        x = self.resnet.forward(x)
        x = F.relu(x)
        x = self.fc1_drop(x)
#         x = F.relu(self.linear2(x))
#         x = self.fc2_drop(x)
        x = self.linear3(x)
        return x

In [105]:
torch.cuda.set_device(1)

In [106]:
NUM_CLASSES = 144

In [107]:
# #Train, test, validation Split
from PIL import Image
import shutil
data_split = False
if data_split:
    DATA_DIR = './Amphibia/train'
    i = 1
    for dir_path,dirs, files in os.walk(DATA_DIR):
        print(i)
        i+=1
        for f in files:
            if f.endswith('.jpg'):
                k = np.random.rand()
                if k <= 0.15:
                    folder = 'test'
                elif k <= 0.3:
                    folder = 'val'
                else:
                    folder = 'train'
                img = Image.open(os.path.join(dir_path, f))
                img = img.resize((224, 224), Image.ANTIALIAS)
                img.save(os.path.join('/Users/ryancheng/Desktop/cs194-129/project/data',folder,dir_path[-4:],f))
    

In [108]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = './data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(dataset_sizes)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

{'train': 8161, 'val': 1754}


In [109]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        epoch_time = time.time()
        print('Epoch {}/{}'.format(epoch+1, num_epochs), end=" ")
        #print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
#             for m in model.modules():
#                 if isinstance(m, nn.BatchNorm2d):
#                     m.eval()
#                 else:
#                     m.train()
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0]*inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc), end=" ") 
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        e_time = time.time() - epoch_time
        print('time: {:.0f}m {:.0f}s'.format(
                e_time // 60, e_time%60))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [110]:
resnet50 = resnet50
criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
resnet_FC = resnet50_3FC(resnet50).cuda()
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet_FC.parameters()), lr=0.01)
# print(list(filter(lambda p: p.requires_grad, resnet_FC.parameters())))
# Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
resnet50 = train_model(resnet_FC, criterion, optimizer_ft, None, num_epochs=50)

Epoch 1/50 train Loss: 4.4876 Acc: 0.0594 val Loss: 5.0387 Acc: 0.0462 time: 0m 50s
Epoch 2/50 train Loss: 4.2399 Acc: 0.0654 val Loss: 4.2037 Acc: 0.0764 time: 0m 52s
Epoch 3/50 train Loss: 4.2002 Acc: 0.0701 val Loss: 4.1602 Acc: 0.0827 time: 0m 54s
Epoch 4/50 train Loss: 4.1707 Acc: 0.0788 val Loss: 4.1267 Acc: 0.0832 time: 0m 54s
Epoch 5/50 train Loss: 4.1324 Acc: 0.0853 val Loss: 4.4065 Acc: 0.0787 time: 0m 54s
Epoch 6/50 train Loss: 4.1204 Acc: 0.0892 val Loss: 4.2281 Acc: 0.0946 time: 0m 55s
Epoch 7/50 train Loss: 4.1088 Acc: 0.0857 val Loss: 4.7015 Acc: 0.0867 time: 0m 54s
Epoch 8/50 train Loss: 4.1130 Acc: 0.0891 val Loss: 4.0514 Acc: 0.0981 time: 0m 55s
Epoch 9/50 train Loss: 4.0840 Acc: 0.0935 val Loss: 4.0452 Acc: 0.1078 time: 0m 55s
Epoch 10/50 train Loss: 4.0766 Acc: 0.0921 val Loss: 4.0510 Acc: 0.0981 time: 0m 55s
Epoch 11/50 train Loss: 4.0635 Acc: 0.0945 val Loss: 4.0690 Acc: 0.0981 time: 0m 55s
Epoch 12/50 train Loss: 4.0532 Acc: 0.0968 val Loss: 4.0324 Acc: 0.0998 ti